In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
df_price = pd.read_excel('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/郵局獲利能力.xlsx')
df_neighbor = pd.read_excel('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/附近郵局數.xlsx')
df_errortime = pd.read_excel('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/異常時間.xlsx')
colnames=['ACC01', '案件數量'] 
df_number = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/郵局案件量.csv', names=colnames, header=None)
colnames=['ACC01', '空值量'] 
df_null = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/空直.csv', names=colnames, header=None)
colnames=['ACC01', '案件沖銷量'] 
df_back = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/back.csv', names=colnames, header=None)

In [6]:
df_neighbor = df_neighbor.rename(columns={'郵務局號': 'ACC01'})
df_neighbor = df_neighbor[['ACC01','附近郵局數']]

In [7]:
df_errortime['交寄時間異常'] = df_errortime['交寄時間異常'].fillna(0)

In [19]:
df_all = pd.merge(df_price, df_neighbor, on='ACC01')
df_all = pd.merge(df_all, df_number, on='ACC01')
df_all = pd.merge(df_all, df_null, on='ACC01')
df_all = pd.merge(df_all, df_back, on='ACC01')
df_all = pd.merge(df_all, df_errortime, on='ACC01')

In [20]:
df_all = df_all.sort_values(by=['ACC01']).reset_index(drop=True)

In [21]:
df_all = df_all.rename(columns={'price': '獲利能力'})

In [22]:
# 特約客戶
df_cc = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/CC.csv', encoding='big5', header=None)

In [23]:
df_customer = df_cc.groupby([8]).size().reset_index()
df_customer = df_customer.rename(columns={0: '特約客戶數量', 8: 'ACC01'})

In [24]:
df_all = pd.merge(df_all, df_customer, on='ACC01', how='left')

In [28]:
df_all['特約客戶數量'] = df_all['特約客戶數量'].fillna(0)

In [29]:
# 計算時間
df_time = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/營業資訊.csv')

In [30]:
df_acc = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/雙北市ACC價錢.csv')
df_acc = df_acc[['局名','ACC01','ACC06','ACC13']]
df_acc = df_acc.groupby(['局名','ACC06'])['ACC13'].sum().reset_index()

/Users/andy/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
# 營業天數
date = []
for i in df_all['局名']:
    date.append(len(df_acc[df_acc['局名']==i]))

In [32]:
df_all = pd.concat([df_all, pd.DataFrame(date, columns=['營業天數'])], axis=1)

In [33]:
# 區域人口數
df_area = pd.read_csv('/Volumes/transcend/大檔案/中華郵政/郵務局號.csv')
df_people = pd.read_excel('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/雙北人口數.xlsx')

In [34]:
df_area = df_area[['郵務局號','鄉鎮市區']]
df_area = df_area.rename(columns={'郵務局號': 'ACC01'})

In [41]:
df_all = pd.merge(df_all, df_area, on='ACC01')
df_all = pd.merge(df_all, df_people, on='鄉鎮市區')
df_all = df_all.rename(columns={'人口數': '區域人口數'})

In [42]:
df_all = df_all[['ACC01','局名','鄉鎮市區','區域人口數','附近郵局數','獲利能力','案件數量','特約客戶數量','營業天數','空值量','交寄時間異常']]

In [43]:
df_all.head()

,ACC01,局名,鄉鎮市區,區域人口數,附近郵局數,獲利能力,案件數量,特約客戶數量,營業天數,空值量,交寄時間異常
0,100001,臺北東門,中正區,158709,8,1700817,122151,27.0,71,145277,693.0
1,100005,臺北南海,中正區,158709,9,2577493,555857,30.0,71,215173,0.0
2,100006,臺北螢橋,中正區,158709,4,989057,53165,18.0,60,75189,13.0
3,100008,臺北復興橋,中正區,158709,11,1095432,176533,25.0,60,102501,10.0
4,100012,行政院,中正區,158709,9,641832,131994,17.0,60,61454,0.0


In [45]:
df_all.to_excel('/Volumes/transcend/大檔案/中華郵政/雷達圖指標/雷達指標表.xlsx')